In [1]:
import os
import cv2
import numpy as np
from scipy.interpolate import interp2d  #deprecated
from scipy.interpolate import RectBivariateSpline

In [2]:
# Set the path to the folder containing the image sequence
folder_path = "./Hotel Sequence/"

# Get a list of all image files in the folder
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".png")]
image_files = sorted(image_files, key=lambda item: (int(item.split("seq")[1].split('.')[0])))

In [3]:
# Read the first image in the sequence
prev_frame = cv2.imread(image_files[0])

# Convert the image to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Detect keypoints in the first frame using the Harris Corner detector
prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=500, qualityLevel=0.01, minDistance=10)

# Select 20 random keypoints in the first frame
np.random.seed(0)
random_pts = np.random.permutation(prev_pts)[:20]

# Initialize an empty list to store the keypoints that move out of the frame
out_of_frame_pts = []

In [4]:
mask = np.zeros_like(prev_frame)
# Loop through the remaining images in the sequence
for i in range(1, len(image_files)):

    # Read the current frame
    frame = cv2.imread(image_files[i])

    # Convert the current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute the optical flow for each keypoint
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(16, 16), maxLevel=2)

    # Select only the keypoints that were successfully tracked
    good_new = p1[st==1]
    good_old = prev_pts[st==1]

    # Initialize a mask to overlay the 2D path of the keypoints
#     mask = np.zeros_like(frame)

    # Loop through each tracked keypoint
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        old = old.astype(int)
        new = new.astype(int)
        
        # Compute the displacement vector
        u, v = new.ravel() - old.ravel()

        # Check if the predicted location is out of the frame or near the image borders
        if new[0] < 0 or new[0] >= frame.shape[1] or new[1] < 0 or new[1] >= frame.shape[0] or new[0] < 15 or new[0] >= frame.shape[1] - 15 or new[1] < 15 or new[1] >= frame.shape[0] - 15:
            # If the keypoint is out of the frame, add it to the list of out-of-frame keypoints
            out_of_frame_pts.append(old)
            # Skip to the next keypoint
            continue

        # Create a window around the keypoint in the previous frame
        x, y = old.ravel().astype(int)
        window = prev_gray[y-8:y+8, x-8:x+8]

        # Compute the interpolated position of the keypoint in the current frame
        f = RectBivariateSpline(np.arange(x-8, x+8), np.arange(y-8, y+8), window)
        new_int = np.array([new[0], new[1]]).reshape(1, 2)
    
        out = f(new_int[:, 0], new_int[:, 1])
        if len(out) == 2:
            u_int, v_int = out
        else:
            u_int = v_int = 0

        # Convert the displacement vector to integer values
        u_int, v_int = int(round(u_int)), int(round(v_int))
                
        # Draw a line segment connecting the previous and current keypoint locations
        mask = cv2.line(mask, tuple(old.ravel()), tuple(new.ravel()), (0, 255, 0), 2)

        # Draw a circle around the current keypoint location
        frame = cv2.circle(frame, tuple(new.ravel()), 5, (0, 0, 255), -1)

    # Overlay the mask on the current frame
    img = cv2.add(frame, mask)
    # Display the image with keypoints and paths
    cv2.imshow("Frame", img)
    cv2.waitKey(2)
    
    # Update the previous frame and keypoints
    prev_gray = gray.copy()
    prev_pts = good_new.reshape(-1, 1, 2)
    
#     # create a seperate image of the first frame
#     out_of_frame_img = cv2.imread(image_files[0])
    
#     for pt in out_of_frame_pts:
#         # Draw a circle around the initial keypoint location
#         out_of_frame_img = cv2.circle(out_of_frame_img, tuple(pt.ravel()), 5, (0, 0, 255), -1)
#         cv2.imshow("Out of Frame", out_of_frame_img)
#         cv2.waitKey(1)
        
cv2.destroyAllWindows()

## IMP: The last few lines have been commented to better visualise the klt, uncomment to see the out of frame pts with respect to first image

## IMP: RERUN ALL THE CELLS AFTER YOU'VE RAN IT ONCE